In [1]:
# data
import os

from nltk.corpus import stopwords

path_root = "C:/BCSpace/study/ThisSemester/IR/lab/lab5"
dir_source = "20_newsgroups"
dir_temp = "_temp"  # folder name for the processed files
file_indexer = 'indexer.csv'    # new file to store the indexer

path_source = os.path.join(path_root, dir_source)
path_temp = os.path.join(path_root, dir_temp)
doc_id = {}
doc_list = []
term_idf = {}
spimi_index = {}

stop_words = {'i', 'me', 'my', 'myself', 'we', 
              'our', 'ours', 'ourselves', 'you', 
              "you're", "you've", "you'll", "you'd", 
              'your', 'yours', 'yourself', 'yourselves', 
              'he', 'him', 'his', 'himself', 'she', 
              "she's", 'her', 'hers', 'herself', 'it', 
              "it's", 'its', 'itself', 'they', 'them', 
              'their', 'theirs', 'themselves', 'what', 
              'which', 'who', 'whom', 'this', 'that', 
              "that'll", 'these', 'those', 'am', 'is', 
              'are', 'was', 'were', 'be', 'been', 
              'being', 'have', 'has', 'had', 'having', 
              'do', 'does', 'did', 'doing', 'a', 'an', 
              'the', 'and', 'but', 'if', 'or', 'because', 
              'as', 'until', 'while', 'of', 'at', 'by', 
              'for', 'with', 'about', 'against', 'between', 
              'into', 'through', 'during', 'before', 'after', 
              'above', 'below', 'to', 'from', 'up', 'down', 
              'in', 'out', 'on', 'off', 'over', 'under', 
              'again', 'further', 'then', 'once', 'here', 
              'there', 'when', 'where', 'why', 'how', 
              'all', 'any', 'both', 'each', 'few',
              'more', 'most', 'other', 'some', 'such', 
              'no', 'nor', 'not', 'only', 'own', 
              'same', 'so', 'than', 'too', 'very', 
              's', 't', 'can', 'will', 'just', 'don', 
              "don't", 'should', "should've", 'now', 'd', 
              'll', 'm', 'o', 're', 've', 'y', 'ain', 
              'aren', "aren't", 'couldn', "couldn't", 
              'didn', "didn't", 'doesn', "doesn't", 'hadn', 
              "hadn't", 'hasn', "hasn't", 'haven', "haven't", 
              'isn', "isn't", 'ma', 'mightn', "mightn't", 
              'mustn', "mustn't", 'needn', "needn't", 'shan', 
              "shan't", 'shouldn', "shouldn't", 'wasn', 
              "wasn't", 'weren', "weren't", 'won', "won't", 
              'wouldn', "wouldn't"}
# stop_words = set(stopwords.words('english'))
stop_words.add('s')


In [2]:
# Some functions
import csv
from math import log10

from nltk import RegexpTokenizer
import sys


# Recursively finds size of objects
def get_size(obj, seen=None):
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size


# tokenize

def get_term_freq(text):
    text = text.lower()
    # word_tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
    word_tokenizer = RegexpTokenizer('[A-Za-z]+')
    words = word_tokenizer.tokenize(text)
    term_set = set(words).difference(stop_words)
    term_freq = {}
    for w in words:
        if w in term_set:
            if w in term_freq:
                term_freq[w] = term_freq[w] + 1
            else:
                term_freq[w] = 1
    return term_freq


# store sub dictionary in .csv
def write_batch(cid, index):
    batch_file = os.path.join(path_temp, str(cid) + '.csv')
    with open(batch_file, mode='w+', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        for key, value in index.items():
            value.sort()
            temp_list = [(key, len(value))] + value
            f_writer.writerow(temp_list)
          
  
def write_indexer(p, index):
    with open(p, mode='w+', newline='', encoding='utf-8') as f:
        f_writer = csv.writer(f)
        for key, value in index.items():
            temp_list = [(key, term_idf[key])] + value
            f_writer.writerow(temp_list)


def read_batch(p):
    temp_index = {}
    with open(p, 'r', encoding='utf-8') as f:
        f_reader = csv.reader(f)
        for row in f_reader:
            if len(row) > 1:
                rlst = []
                for tu in row:
                    rlst.append(tuple(eval(tu)))
                temp_index[rlst[0][0]] = rlst[1:]
    return temp_index 
    

def read_indexer(p):
    temp_index = {}
    temp_idf = {}
    with open(p, 'r', encoding='utf-8') as f:
        f_reader = csv.reader(f)
        for row in f_reader:
            if len(row) > 1:
                rlst = []
                for tu in row:
                    rlst.append(tuple(eval(tu)))
                temp_index[rlst[0][0]] = rlst[1:]
                temp_idf[rlst[0][0]] = rlst[0][1]
    return temp_index, temp_idf


def get_w(tf, idf):
    if idf == 0 or tf == 0:
        return 0
    return (1 + log10(tf)) * log10(idf)


In [3]:
# Calculate the indexer and store into 'indexer.csv'

# get the sub dictionaries
from sys import getsizeof
from math import sqrt

chunk_size = 15000000


def process_directions():
    chunk_num = 0
    term_index = {}
    doc_list.clear()
    doc_id.clear()
    if not os.path.exists(path_temp):
        os.mkdir(path_temp)
    for root, dirs, files in os.walk(path_source):
        for file_name in files:
            fpn = os.path.join(root, file_name)
            doc_list.append([fpn, 0])
            file_cnt = len(doc_list) - 1
            doc_id[fpn] = file_cnt
            
            with open(os.path.join(root, fpn), mode='r', errors='ignore', encoding='utf-8') as f:
                text = f.read()
            term_freq = get_term_freq(text)
            for term, value in term_freq.items():
                if term in term_index:
                    term_index[term].append((file_cnt, value))
                else:
                    term_index[term] = [(file_cnt, value)]
            if file_cnt % 100 == 0 and get_size(term_index) > chunk_size:
                print(str(chunk_num) + '.csv' + ' created')
                write_batch(chunk_num, term_index)
                term_index.clear()
                chunk_num = chunk_num + 1
    if getsizeof(term_index) > 0:
        print(str(chunk_num) + '.csv' + ' created')
        write_batch(chunk_num, term_index)
        chunk_num = chunk_num + 1
    return chunk_num


# merge index
def merge_batch():
    spimi_index.clear()
    term_idf.clear()
    for i in range(batch_num):
        file = str(i) + '.csv'
        file_name = os.path.join(path_temp, file)
        sub_index = read_batch(file_name)
        print(file + ' merged')
        for key, value in sub_index.items():
            if key in spimi_index:
                spimi_index[key] = spimi_index[key] + value
            else:
                spimi_index[key] = value
    file_num = len(doc_list)
    for key, value in spimi_index.items():
        term_idf[key] = float(file_num) / len(value)
    write_indexer(os.path.join(path_root, file_indexer), spimi_index)


def trans_w():
    for key, value in spimi_index.items():
        temp_list = value
        for i in range(len(value)):
            temp_list[i] = (temp_list[i][0], get_w(temp_list[i][1], term_idf[key]))
            doc_list[temp_list[i][0]][1] = doc_list[temp_list[i][0]][1] + temp_list[i][1] * temp_list[i][1]
        spimi_index[key] = temp_list
    for i in range(len(doc_list)):
        doc_list[i][1] = sqrt(doc_list[i][1])


In [4]:
# calculate the indexer
batch_num = process_directions()
merge_batch()

trans_w()

# Load indexer from file

# spimi_index, term_idf = read_indexer(os.path.join(path_root, file_indexer))
# trans_w()


0.csv created


1.csv created


2.csv created


3.csv created


4.csv created


...

13.csv merged


14.csv merged


15.csv merged


16.csv merged


17.csv merged


In [6]:
# Query
query_list = ['handwriting signature',
              'Georgia Institute Technology',
              'evidence piling',
              'Information retrieval',
              'Los Angeles',
              'ugce'] 
query = query_list[5]

query_freq = get_term_freq(query)
candi_dict = {}

for term, tfq in query_freq.items():
    w = get_w(tfq, term_idf[term])
    query_freq[term] = w

for term in query_freq:
    for fid, tf_idf in spimi_index[term]:
        if fid in candi_dict:
            temp_tuple = candi_dict[fid]
            candi_dict[fid] = tf_idf * query_freq[term] + temp_tuple
        else:
            candi_dict[fid] = tf_idf * query_freq[term]

candi_list = []
for key, value in candi_dict.items():
    candi_list.append((value / doc_list[key][1], key))

candi_list.sort(reverse=True, key=lambda x: (x[0], -x[1]))

for w, fid in candi_list[:10]:
    print(doc_list[fid][0][len(path_root):])



\20_newsgroups\comp.os.ms-windows.misc\9981
